<a href="https://colab.research.google.com/github/JTStephens18/XCubeImplementation/blob/main/XCube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install torchinfo

In [2]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchinfo import summary

# Binvox Processor

In [3]:
#  Copyright (C) 2012 Daniel Maturana
#  This file is part of binvox-rw-py.
#
#  binvox-rw-py is free software: you can redistribute it and/or modify
#  it under the terms of the GNU General Public License as published by
#  the Free Software Foundation, either version 3 of the License, or
#  (at your option) any later version.
#
#  binvox-rw-py is distributed in the hope that it will be useful,
#  but WITHOUT ANY WARRANTY; without even the implied warranty of
#  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#  GNU General Public License for more details.
#
#  You should have received a copy of the GNU General Public License
#  along with binvox-rw-py. If not, see <http://www.gnu.org/licenses/>.
#

"""
Binvox to Numpy and back.


>>> import numpy as np
>>> import binvox_rw
>>> with open('chair.binvox', 'rb') as f:
...     m1 = binvox_rw.read_as_3d_array(f)
...
>>> m1.dims
[32, 32, 32]
>>> m1.scale
41.133000000000003
>>> m1.translate
[0.0, 0.0, 0.0]
>>> with open('chair_out.binvox', 'wb') as f:
...     m1.write(f)
...
>>> with open('chair_out.binvox', 'rb') as f:
...     m2 = binvox_rw.read_as_3d_array(f)
...
>>> m1.dims==m2.dims
True
>>> m1.scale==m2.scale
True
>>> m1.translate==m2.translate
True
>>> np.all(m1.data==m2.data)
True

>>> with open('chair.binvox', 'rb') as f:
...     md = binvox_rw.read_as_3d_array(f)
...
>>> with open('chair.binvox', 'rb') as f:
...     ms = binvox_rw.read_as_coord_array(f)
...
>>> data_ds = binvox_rw.dense_to_sparse(md.data)
>>> data_sd = binvox_rw.sparse_to_dense(ms.data, 32)
>>> np.all(data_sd==md.data)
True
>>> # the ordering of elements returned by numpy.nonzero changes with axis
>>> # ordering, so to compare for equality we first lexically sort the voxels.
>>> np.all(ms.data[:, np.lexsort(ms.data)] == data_ds[:, np.lexsort(data_ds)])
True
"""

import numpy as np

class Voxels(object):
    """ Holds a binvox model.
    data is either a three-dimensional numpy boolean array (dense representation)
    or a two-dimensional numpy float array (coordinate representation).

    dims, translate and scale are the model metadata.

    dims are the voxel dimensions, e.g. [32, 32, 32] for a 32x32x32 model.

    scale and translate relate the voxels to the original model coordinates.

    To translate voxel coordinates i, j, k to original coordinates x, y, z:

    x_n = (i+.5)/dims[0]
    y_n = (j+.5)/dims[1]
    z_n = (k+.5)/dims[2]
    x = scale*x_n + translate[0]
    y = scale*y_n + translate[1]
    z = scale*z_n + translate[2]

    """

    def __init__(self, data, dims, translate, scale, axis_order):
        self.data = data
        self.dims = dims
        self.translate = translate
        self.scale = scale
        assert (axis_order in ('xzy', 'xyz'))
        self.axis_order = axis_order

    def clone(self):
        data = self.data.copy()
        dims = self.dims[:]
        translate = self.translate[:]
        return Voxels(data, dims, translate, self.scale, self.axis_order)

    def write(self, fp):
        write(self, fp)

def read_header(fp):
    """ Read binvox header. Mostly meant for internal use.
    """
    line = fp.readline().strip()
    if not line.startswith(b'#binvox'):
        raise IOError('Not a binvox file')
    dims = list(map(int, fp.readline().strip().split(b' ')[1:]))
    translate = list(map(float, fp.readline().strip().split(b' ')[1:]))
    scale = list(map(float, fp.readline().strip().split(b' ')[1:]))[0]
    line = fp.readline()
    return dims, translate, scale

def read_as_3d_array(fp, fix_coords=True):
    """ Read binary binvox format as array.

    Returns the model with accompanying metadata.

    Voxels are stored in a three-dimensional numpy array, which is simple and
    direct, but may use a lot of memory for large models. (Storage requirements
    are 8*(d^3) bytes, where d is the dimensions of the binvox model. Numpy
    boolean arrays use a byte per element).

    Doesn't do any checks on input except for the '#binvox' line.
    """
    dims, translate, scale = read_header(fp)
    raw_data = np.frombuffer(fp.read(), dtype=np.uint8)
    # if just using reshape() on the raw data:
    # indexing the array as array[i,j,k], the indices map into the
    # coords as:
    # i -> x
    # j -> z
    # k -> y
    # if fix_coords is true, then data is rearranged so that
    # mapping is
    # i -> x
    # j -> y
    # k -> z
    values, counts = raw_data[::2], raw_data[1::2]
    data = np.repeat(values, counts).astype(np.bool)
    data = data.reshape(dims)
    if fix_coords:
        # xzy to xyz TODO the right thing
        data = np.transpose(data, (0, 2, 1))
        axis_order = 'xyz'
    else:
        axis_order = 'xzy'
    return Voxels(data, dims, translate, scale, axis_order)

def read_as_coord_array(fp, fix_coords=True):
    """ Read binary binvox format as coordinates.

    Returns binvox model with voxels in a "coordinate" representation, i.e.  an
    3 x N array where N is the number of nonzero voxels. Each column
    corresponds to a nonzero voxel and the 3 rows are the (x, z, y) coordinates
    of the voxel.  (The odd ordering is due to the way binvox format lays out
    data).  Note that coordinates refer to the binvox voxels, without any
    scaling or translation.

    Use this to save memory if your model is very sparse (mostly empty).

    Doesn't do any checks on input except for the '#binvox' line.
    """
    dims, translate, scale = read_header(fp)
    raw_data = np.frombuffer(fp.read(), dtype=np.uint8)

    values, counts = raw_data[::2], raw_data[1::2]

    sz = np.prod(dims)
    index, end_index = 0, 0
    end_indices = np.cumsum(counts)
    indices = np.concatenate(([0], end_indices[:-1])).astype(end_indices.dtype)

    values = values.astype(np.bool)
    indices = indices[values]
    end_indices = end_indices[values]

    nz_voxels = []
    for index, end_index in zip(indices, end_indices):
        nz_voxels.extend(range(index, end_index))
    nz_voxels = np.array(nz_voxels)
    # TODO are these dims correct?
    # according to docs,
    # index = x * wxh + z * width + y; // wxh = width * height = d * d

    x = nz_voxels / (dims[0]*dims[1])
    zwpy = nz_voxels % (dims[0]*dims[1]) # z*w + y
    z = zwpy / dims[0]
    y = zwpy % dims[0]
    if fix_coords:
        data = np.vstack((x, y, z))
        axis_order = 'xyz'
    else:
        data = np.vstack((x, z, y))
        axis_order = 'xzy'

    #return Voxels(data, dims, translate, scale, axis_order)
    return Voxels(np.ascontiguousarray(data), dims, translate, scale, axis_order)

def dense_to_sparse(voxel_data, dtype=np.int):
    """ From dense representation to sparse (coordinate) representation.
    No coordinate reordering.
    """
    if voxel_data.ndim!=3:
        raise ValueError('voxel_data is wrong shape; should be 3D array.')
    return np.asarray(np.nonzero(voxel_data), dtype)

def sparse_to_dense(voxel_data, dims, dtype=np.bool):
    if voxel_data.ndim!=2 or voxel_data.shape[0]!=3:
        raise ValueError('voxel_data is wrong shape; should be 3xN array.')
    if np.isscalar(dims):
        dims = [dims]*3
    dims = np.atleast_2d(dims).T
    # truncate to integers
    xyz = voxel_data.astype(np.int)
    # discard voxels that fall outside dims
    valid_ix = ~np.any((xyz < 0) | (xyz >= dims), 0)
    xyz = xyz[:,valid_ix]
    out = np.zeros(dims.flatten(), dtype=dtype)
    out[tuple(xyz)] = True
    return out

#def get_linear_index(x, y, z, dims):
    #""" Assuming xzy order. (y increasing fastest.
    #TODO ensure this is right when dims are not all same
    #"""
    #return x*(dims[1]*dims[2]) + z*dims[1] + y

def write(voxel_model, fp):
    """ Write binary binvox format.

    Note that when saving a model in sparse (coordinate) format, it is first
    converted to dense format.

    Doesn't check if the model is 'sane'.

    """
    if voxel_model.data.ndim==2:
        # TODO avoid conversion to dense
        dense_voxel_data = sparse_to_dense(voxel_model.data, voxel_model.dims)
    else:
        dense_voxel_data = voxel_model.data

    fp.write('#binvox 1\n')
    fp.write('dim '+' '.join(map(str, voxel_model.dims))+'\n')
    fp.write('translate '+' '.join(map(str, voxel_model.translate))+'\n')
    fp.write('scale '+str(voxel_model.scale)+'\n')
    fp.write('data\n')
    if not voxel_model.axis_order in ('xzy', 'xyz'):
        raise ValueError('Unsupported voxel model axis order')

    if voxel_model.axis_order=='xzy':
        voxels_flat = dense_voxel_data.flatten()
    elif voxel_model.axis_order=='xyz':
        voxels_flat = np.transpose(dense_voxel_data, (0, 2, 1)).flatten()

    # keep a sort of state machine for writing run length encoding
    state = voxels_flat[0]
    ctr = 0
    for c in voxels_flat:
        if c==state:
            ctr += 1
            # if ctr hits max, dump
            if ctr==255:
                fp.write(chr(state))
                fp.write(chr(ctr))
                ctr = 0
        else:
            # if switch state, dump
            fp.write(chr(state))
            fp.write(chr(ctr))
            state = c
            ctr = 1
    # flush out remainders
    if ctr > 0:
        fp.write(chr(state))
        fp.write(chr(ctr))


    import doctest
    doctest.testmod()

<ipython-input-3-ef32016758ed>:203: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def dense_to_sparse(voxel_data, dtype=np.int):
<ipython-input-3-ef32016758ed>:211: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def sparse_to_de

# Experiments

In [4]:
!unzip "/content/drive/MyDrive/ShapeNet_Chair_SolidVoxel/binvoxModels.zip"

Streaming output truncated to the last 5000 lines.
  inflating: binvoxModels/4b95e968966fafc6e0675251723e1e08.binvox  
  inflating: binvoxModels/4b9ccf80a846432095bea7c29e873d16.binvox  
  inflating: binvoxModels/4bb5877100a76cf5bee7d080c8f1e1fd.binvox  
  inflating: binvoxModels/4bbd110ccfc81dd336b0f2a1430e993a.binvox  
  inflating: binvoxModels/4bc064672eb85023d84a8130bee3aae8.binvox  
  inflating: binvoxModels/4bc5920f74bbf6412acd2bb6205825cb.binvox  
  inflating: binvoxModels/4bc5a889b3ef967b9de7cc399bc9b2b3.binvox  
  inflating: binvoxModels/4bcc7c4e4c0bfe47e8f8c11a24c52ebb.binvox  
  inflating: binvoxModels/4bcf02d408a42fedc399b0e96597941f.binvox  
  inflating: binvoxModels/4bd7a49e63e0e97936e3b2fa8d1eb4eb.binvox  
  inflating: binvoxModels/4bdbecfbc925219157915a20ae9ec6b6.binvox  
  inflating: binvoxModels/4c0983329afcd06f730e89ca0d2d13c3.binvox  
  inflating: binvoxModels/4c0d8f04f27f96284417bea8868af4d1.binvox  
  inflating: binvoxModels/4c1777173111f2e380a88936375f2ef4.binvox

In [5]:
# Should have train/test/val split - 80% train / 20% test/val
# Have a dictionary where {'train': ["id-1",  "id-2"], 'validation': ["id-3", "id-4", 'id-5']}
# labels = {'id-1': 0, "id-2": 1, "id-3": 0, "id-4": 1, "id-5": 2}
# Allows me to access a data point through a specific index

partition = {"train": [], "validation": []}
labels = {}

binvoxFiles = os.listdir("./binvoxModels")
# 20% of data size
splitSize = int(len(binvoxFiles) * 0.2)
# Randomly choose files to be in validation set
randomIdx = random.sample(range(len(binvoxFiles)), splitSize)

print(binvoxFiles[:5])

for i, binvox in enumerate(binvoxFiles):
  # with open(f'./binvoxModels/{binvox}', 'rb') as f:
  #   model = read_as_3d_array(f)

  # data = torch.from_numpy(model.data.astype(float)).unsqueeze(0).float()

  if(i in randomIdx):
    labels[binvox.split(".")[0]] = len(partition["validation"])
    # partition["validation"].append(data)
    partition["validation"].append(binvox)
  else:
    labels[binvox.split(".")[0]] = len(partition["train"])
    # partition["train"].append(data)
    partition["train"].append(binvox)

['89133468bce0e63f8101accd22c701b9.binvox', '6df81788b1cdd1c3ab93f6188b226527.binvox', '8a5d60067de905336c183a120a388982.binvox', 'c54a464d63efcab2c389b3ea958c8248.binvox', '648fc7caefda981543d890f2dbb65163.binvox']


## Binvox Tensor Tests

In [ ]:
binvoxFiles = os.listdir("./binvoxModels")
print(binvoxFiles[:5])

['4cfc78be98de1d2bd607f0b8430fb29a.binvox', '3ede0a86fff6f79632284c722d808bec.binvox', '7fe836b46890d24a65ba78ad9601cf1b.binvox', 'b42953e6e55ba5d4d2bc32977ed52527.binvox', 'e39df7339552595febad4f49b26ec52.binvox']


In [ ]:
binvoxTensors = os.listdir("./binvoxTensors")
for i, binvox in enumerate(binvoxFiles[1500:2500]):
  binvoxFile = binvox.split(".")
  if(f'{binvoxFile}.pt' not in binvoxTensors):
    with open(f'./binvoxModels/{binvox}', 'rb') as f:
      model = read_as_3d_array(f)

    data = torch.from_numpy(model.data.astype(float)).unsqueeze(0).float()
    torch.save(data, f'./binvoxTensors/{binvox}.pt')

<ipython-input-5-ef32016758ed>:143: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.repeat(values, counts).astype(np.bool)


In [ ]:
print(len(partition["validation"]))
print(partition["validation"][0].shape)

1355
torch.Size([1, 128, 128, 128])


In [ ]:
! zip "./binvoxTensors.zip" "./binvoxTensors"

In [ ]:
! cp binvoxTensors.zip /content/drive/MyDrive/ShapeNet_Chair_SolidVoxel/BinvoxTensors

## Dataset / Loader

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
class VoxelDataset(Dataset):
  def __init__(self, data):
    self.samples = len(data)
    self.data = data

  def __len__(self):
    return self.samples

  def __getitem__(self, index):
    sample = self.data[index]
    return sample

In [8]:
training_set = VoxelDataset(partition["train"])
train_loader = torch.utils.data.DataLoader(training_set, batch_size=16, num_workers=4, shuffle=True)

In [9]:
class Encoder(nn.Module):
  def __init__(self, in_size, trg_size, device):
    super(Encoder, self).__init__()

  # Initial downsampling layer to convert 128^3 into 16^3
    self.conv_layer_1 = self._conv_layer_set(1, 1, 4, 2, 1, maxPool=True)
    # self.conv_layer_2 = self._conv_layer_set(1,1,1,2,0)
    # self.conv_layer_2 = self._conv_layer_set(trg_size*8, trg_size*4, 4, 2, 1)
    # self.conv_layer_3 = self._conv_layer_set(trg_size*4, trg_size*2, 4, 2, 1)
    # self.conv_layer_4 = self._conv_layer_set(trg_size*2, trg_size, 3, 2, 1)

# Set of convolutions to convert bottleneck tensor into latent tensor of the same shape
    self.latent_conv_1 = self._conv_transpose_layer_set(1, 64, 4, 2, 1)
    self.latent_conv_2 = self._conv_transpose_layer_set(64, 128, 4, 2, 1)
    self.latent_conv_3 = self._conv_layer_set(128, 64, 4, 2, 1, maxPool=False)
    self.latent_conv_4 = self._conv_layer_set(64, 1, 4, 2, 1, maxPool=False)
    self.flatten = nn.Flatten()

  # 16^3 = 4096
    self.z_mean = nn.Linear(4096, 3)
    self.z_log_var = nn.Linear(4096, 3)


    self.decoder = nn.Sequential(
        nn.Linear(3, 4096)
    )
    self.device = device


  def _conv_layer_set(self, in_c, out_c, kernel_size, stride, padding, maxPool):
    if(maxPool == True):
      conv_layer = nn.Sequential(
          nn.Conv3d(in_c, out_c, kernel_size, stride, padding),
          nn.MaxPool3d(1, kernel_size)
      )
    else:
      conv_layer = nn.Sequential(
          nn.Conv3d(in_c, out_c, kernel_size, stride, padding)
      )
    return conv_layer


  def _conv_transpose_layer_set(self, in_c, out_c, kernel_size, stride, padding):
    conv_layer = nn.Sequential(
        nn.ConvTranspose3d(in_c, out_c, kernel_size, stride, padding)
    )
    return conv_layer


  def reparameterize(self, z_mu, z_log_var):
    eps = torch.randn(z_mu.size(0), z_mu.size(1)).to(self.device)
    z = z_mu + eps * torch.exp(z_log_var/2.).to(self.device)
    return z


  def forward(self, x):
    # bn = bottleneck
    bn = self.conv_layer_1(x)

    out = self.latent_conv_1(bn)
    out = self.latent_conv_2(out)
    out = self.latent_conv_3(out)
    out = self.latent_conv_4(out)
    latent_vec = self.flatten(out)
    z_mean, z_log_var = self.z_mean(latent_vec), self.z_log_var(latent_vec)
    z = self.reparameterize(z_mean, z_log_var)
    # Z is the sampled vector from the mean and sd.
    # Could be interesting to instead upsample z into 16^3 and use that as the input to maskingModule instead of out
    z = self.decoder(z)
    z = z.view(x.shape[0], 1, 16, 16, 16)
    return out, z_mean, z_log_var, z

In [23]:
testTensorList = []

encoder = Encoder(128, 16, device).to(device)
# input = torch.from_numpy(newModel).unsqueeze(0).float()
for i in range(2):
  with open(f'./binvoxModels/{partition["train"][i]}', 'rb') as f:
    model = read_as_3d_array(f)
  newModel = torch.from_numpy(model.data.astype(float)).unsqueeze(0).float().to(device)
  testTensorList.append(newModel)
testInputTensor = torch.stack(testTensorList, dim=0).to(device)
# print(input.shape)
# input = input.float()
output, z_mean, z_log_var, z = encoder(testInputTensor)

<ipython-input-3-ef32016758ed>:143: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.repeat(values, counts).astype(np.bool)


In [34]:
print(output.shape)
print(z_mean)
print(z_log_var)
print(z.shape)
# print(output[0][0][6])
# print(z[0][0][6])

torch.Size([2, 1, 16, 16, 16])
tensor([[-0.0142, -0.0105,  0.0008],
        [-0.0142, -0.0105,  0.0008]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([[ 0.0042, -0.0013, -0.0159],
        [ 0.0042, -0.0014, -0.0159]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
torch.Size([2, 1, 16, 16, 16])


In [ ]:
summary(encoder, input_size=(1, 128, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
Encoder                                  [1, 16, 16, 16]           --
├─Sequential: 1-1                        [1, 16, 16, 16]           --
│    └─Conv3d: 2-1                       [1, 64, 64, 64]           65
│    └─MaxPool3d: 2-2                    [1, 16, 16, 16]           --
├─Sequential: 1-2                        [64, 32, 32, 32]          --
│    └─ConvTranspose3d: 2-3              [64, 32, 32, 32]          4,160
├─Sequential: 1-3                        [128, 64, 64, 64]         --
│    └─ConvTranspose3d: 2-4              [128, 64, 64, 64]         524,416
├─Sequential: 1-4                        [64, 32, 32, 32]          --
│    └─Conv3d: 2-5                       [64, 32, 32, 32]          524,352
├─Sequential: 1-5                        [1, 16, 16, 16]           --
│    └─Conv3d: 2-6                       [1, 16, 16, 16]           4,097
├─Flatten: 1-6                           [1, 4096]                 --

In [ ]:
print(output[0][10][4])

tensor([ 0.0009,  0.0006,  0.0007,  0.0007,  0.0007,  0.0007,  0.0007,  0.0006,
         0.0006, -0.0002,  0.0006,  0.0007,  0.0007,  0.0007,  0.0007,  0.0030],
       grad_fn=<SelectBackward0>)


1. Take the predicted output from encoder
2. Calculate normals and determine if it should be subdivided. If yes, subdivide that voxel into an octree
3. If not, determine whether the voxel should be kept or deleted based on masking module prediction
4. Repeat for all voxels on that layer
5. Once finished traversing that layer, move down to the next (the ones that were subdivided) and repeat steps 2-5 until max depth is reached.

Also important to note: If voxels on a lower (AKA finer/subdivided) level are deleted then voxels in the position of the layer above (coarser) replace them

Unsure if this order is correct. Makes more sense to me to determine if something should be kept or deleted before running through each voxel and trying to determine if it should be subdivided.

After some more reading in the XCube paper, I am unsure if we need to use the normal criterion for deciding whether to subdivide. I think we can try to take the output from the masking module and if it is kept then we subdivide and delete if not.

In this vain, we should start by predicting if we keep or delete and then subdivide into the next level based on predictions.

In [10]:
# Takes value from a 3D point and outputs a probability of if it should be kept or pruned
# > 0.5 = kept
# If kept, it is a leaf node
class MaskingModule(nn.Module):
  def __init__(self):
    super(MaskingModule, self).__init__()

    self.fc1 = nn.Linear(1, 8)
    self.fc2 = nn.Linear(8, 1)
    self.ReLU = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.ReLU(out)
    out = self.fc2(out)
    out = self.ReLU(out)
    out = self.sigmoid(out)
    return out

In [11]:
class VAE(nn.Module):
  def __init__(self, MaskingModule, Encoder, device):
    super(VAE, self).__init__()

    self.MaskingModule = MaskingModule
    self.Encoder = Encoder
    self.device = device

  def forward(self, x):
    output, z_mean, z_log_var, z = self.Encoder(x)
    predTensors = torch.zeros(output.shape[0], 1, 128, 128, 128).to(device)
    for binvoxModel in range(output.shape[0]):
      for i in range(output.shape[2]):
        for j in range(output.shape[3]):
          for k in range(output.shape[4]):
            voxel = output[binvoxModel][0][i,j,k]
            pred = self.MaskingModule(voxel.unsqueeze(0))
            if (pred > 0.5):
              subDivVal = voxel / 8.0
              xMin = i*8
              yMin = j*8
              zMin = k*8
              for v in range(8):
                subDivPredInput = subDivVal + (subDivVal*v)
                subDivPred = self.MaskingModule(subDivPredInput.unsqueeze(0))
                if(subDivPred > 0.5):
                  predTensors[binvoxModel][0][xMin+v, yMin+v, zMin+v] = subDivPred
    return predTensors, z_mean, z_log_var, output

In [12]:
vae = VAE(MaskingModule(), Encoder(128, 16, device), device).to(device)

In [27]:
maskingModule = MaskingModule().to(device)
encoder = Encoder(128, 16, device).to(device)
# print(output[0][1][2][3])
# val = output[0][2][7][3].unsqueeze(0)
# print(val)
# print(maskingModule(val))
# predTest = maskingModule(val)

# with open(f'./binvoxModels/{partition["validation"][0]}', 'rb') as f:
#   model = read_as_3d_array(f)
# newModel = torch.from_numpy(model.data.astype(float)).unsqueeze(0).float().to(device)
# newModel = newModel.unsqueeze(0)
# output, z_mean, z_log_var, z = vae(newModel)
# print(z_mean)

In [ ]:
tensorList = []
for i, item in enumerate(next(iter(train_loader))):
  print(i, item)
  with open(f'./binvoxModels/{item}', 'rb') as f:
    model = read_as_3d_array(f)
  newModel = torch.from_numpy(model.data.astype(float)).unsqueeze(0).float()
  tensorList.append(newModel)

finalTensor = torch.stack(tensorList, dim=0)

0 590ae1748c5acdedb05550de345b6d0a.binvox
1 cd9e22cfd389e5d9d330ae3d046a415c.binvox
2 f1dac1909107c0eef51f77a6d7299806.binvox
3 1f5a2c231265aa9380b3cfbeccfb24d2.binvox
4 c56dcaf6b862a6329158e0f216b27548.binvox
5 323bae92774d8def78aac1ce4ecb73b6.binvox
6 b6b911aecb417295e8f8c11a24c52ebb.binvox
7 56fc424a89bb137bf2197da8dec8488d.binvox
8 2724c02bc7797bd1486150a9b7f2cf18.binvox
9 2b454a3d18d5efba615debb484b4c30.binvox
10 aa154024474ad7d138a720835dbeef20.binvox
11 47f0e1452ea20e4856c07129c79effea.binvox
12 9542f23e526bd7aa24adc0b4355a9827.binvox


<ipython-input-4-ef32016758ed>:143: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.repeat(values, counts).astype(np.bool)


13 e9a8c9067bc733ed95bea7c29e873d16.binvox
14 1953d62a63721f17df6cfab91d65bb91.binvox
15 b80e1766b45076ff492d9da2668ec34c.binvox


In [ ]:
print(finalTensor.shape)

torch.Size([16, 1, 128, 128, 128])


In [ ]:
output, z_mean, z_log_var, z = encoder(finalTensor)

In [ ]:
for i, data in enumerate(next(iter(train_loader))):
  count = 0
  print(data)
  for j in range(len(data)):
    count += 1


print(count)

2182398f0f8fdd81af7c7ad2549a1b15.binvox
5a52b62f564eb7e117b431cae0dd70ed.binvox
9dc454ec0b5b7b50e45ef4135c266a12.binvox
8dccb87a61e565cfd4713685ae50fa13.binvox
9b9a114cfee79bfb492d9da2668ec34c.binvox
60167ae64899f8ae966a40fb39b34846.binvox
b654fef8eb98e99d65ba78ad9601cf1b.binvox
40567b0166658623b80ea7d1a9683df8.binvox
c12ea730ea29f8945914d57d976758c0.binvox
b625936e3db6af6318df1fa50d2b64c.binvox
a808d40b66123c7ae9602306ed19eab0.binvox
a10370a1cf036ef82d4725b469da72d.binvox
81c291ab8abc1d0172b24bdfca058442.binvox
70be8ecf5200fddd818fffcf4a59bbc9.binvox
53ea833512f2235319fb4103277a6b93.binvox
3ae2857e04641093b0957d845ac33749.binvox
39


In [13]:
learning_rate = 0.0001
optimizer = torch.optim.Adam(vae.parameters(), learning_rate)

In [14]:
epochs = 100
step = 0
loss_list = []
bceLoss_list = []
kl_div_list = []
for epoch in range(epochs):
  print("***************** Epoch: ", epoch, "***********************")
  for idx, data in enumerate(train_loader):
    step += 1
    # print(idx, data)
    # predTensors = torch.zeros(len(data), 1, 128, 128, 128).to(device)
    tempTensorList = []
    idx+=1
    for item in range(len(data)):
      with open(f"./binvoxModels/{data[item]}", 'rb') as f:
        model = read_as_3d_array(f)
      convertedModel = torch.from_numpy(model.data.astype(float)).unsqueeze(0).float()
      tempTensorList.append(convertedModel)
    inputTensor = torch.stack(tempTensorList, dim=0).to(device)

    outputTensor, z_mean, z_log_var, z = vae(inputTensor)
    # outputVal, z_mean, z_log_var, z = encoder(inputTensor)
    # output = z

    # For each model in the batch: loop through each XYZ index and get the voxel at that position. Predict whether it should be kept or deleted.
    # If kept and on the first layer, then we subdivide to transform 16^3 into 128^3
    # Move to subdivided layer and calculate predictions based on 1/8th of the parent voxel's value - could subdivide even further here, but don't need to in this case.
    # for binvoxModel in range(output.shape[0]):
    #   for i in range(output.shape[2]):
    #     for j in range(output.shape[3]):
    #       for k in range(output.shape[4]):
    #         voxel = output[binvoxModel][0][i,j,k]
    #         pred = maskingModule(voxel.unsqueeze(0))
    #         if (pred > 0.5):
    #           subDivVal = voxel / 8
    #           xMin = i*8
    #           yMin = j*8
    #           zMin = k*8
    #           for v in range(8):
    #             subDivPredInput = subDivVal + (subDivVal*v)
    #             subDivPred = maskingModule(subDivPredInput.unsqueeze(0))
    #             if(subDivPred > 0.5):
    #               predTensors[binvoxModel][0][xMin+v, yMin+v, zMin+v] = subDivPred



    # Loss calculation between predTensors and inputTensor
    bceLoss = nn.BCELoss()
    bceResult = bceLoss(inputTensor, outputTensor)
    # bceResult = bceLoss(inputTensor, predTensors)
    kl_log_term = 1 + z_log_var - z_mean**2 - torch.exp(z_log_var)
    kl_div = -0.5 * torch.sum(kl_log_term, axis=1) - 0.5* torch.sum(torch.logsumexp(kl_log_term, axis=1))
    # kl_div = -0.5 * torch.sum(1 + z_log_var - z_mean**2 - torch.exp(z_log_var), axis=1)
    kl_div = kl_div.mean() # Avg over batch dimension
    kl_weight = 0.0015
    loss = bceResult + (kl_weight * kl_div)

    # loss_list.append(loss.item())
    # bceLoss_list.append(bceResult.item())
    # kl_div_list.append(kl_weight * kl_div)
    print("Loss", loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    torch.save(vae.state_dict(), 'VAECheckpoint.pth')

    # print("Epoch", epoch, "step", step, "Loss", loss, "BCE", bceResult, "KL_Div", (kl_weight*kl_div))
    if step % 50 == 0:
      print("Epoch", epoch, "step", step, "Loss", loss, "BCE", bceResult, "KL_Div", (kl_weight*kl_div))

***************** Epoch:  0 ***********************


<ipython-input-3-ef32016758ed>:143: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.repeat(values, counts).astype(np.bool)


KeyboardInterrupt: 

In [ ]:
! cp VAECheckpoint.pth /content/drive/MyDrive/ShapeNet_Chair_SolidVoxel/VAECheckpoint.pth

In [ ]:
bceLoss = nn.BCELoss()
bceval = bceLoss(newModel, output)
print(bceval)

print(z_log_var[0])
print(z_mean[0])
kl_div = -0.5 * torch.sum(1 + z_log_var - z_mean**2 - torch.exp(z_log_var), dim=1)
#kl_div = kl_div.mean() # Avg over batch dimension

print(kl_div)

ValueError: Using a target size (torch.Size([1, 16, 16, 16])) that is different to the input size (torch.Size([1, 128, 128, 128])) is deprecated. Please ensure they have the same size.

In [ ]:
newTensor = torch.zeros(1, 128, 128, 128)
print(newTensor.shape)
a = 0
b = 0
for i in range(output.shape[1]):
  for j in range(output.shape[2]):
    for k in range(output.shape[3]):
      voxel = output[0][i,j,k]
      pred = maskingModule(voxel.unsqueeze(0))
      if(pred > 0.5):
        # Subdivide into octants
        # Range on axis 1 = [i*8, i+1*8]
        # Additional for loop in range of 8 to add (voxel / 8) + to the index
        subDivVal = voxel / 8
        xMin = i*8
        yMin = j*8
        zMin = k*8
        for v in range(8):
          # Take 8th of the value from the original voxel and predict if we keep or prune it
          subDivPredInput = subDivVal + (subDivVal*v)
          subDivPred = maskingModule(subDivPredInput.unsqueeze(0))
          if(subDivPred > 0.5):
            newTensor[0][xMin+v, yMin+v, zMin+v] = subDivPred
        a += 1
      elif (pred <= 0.5):
        # Delete / Set to 0
        b += 1

torch.Size([1, 128, 128, 128])


In [ ]:
print(a)
print(b)
print(newTensor[0][5][50])

4096
0
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<SelectBackward0>)


## Training Loop

In [ ]:
dataset = VoxelDataset(newModel)

In [ ]:
class OctreeNode():
  def __init__(self, center, size):
    self.center = center
    self.size = size
    self.children = [None] * 8
    self.value = None

def createOctree(data, level, min_size, node_center, node_size):
  if level == 0 or node_size <= min_size:
    # Create a leaf node
    leaf_node = OctreeNode(node_center, node_size)
    leaf_node.value = data[node_center]
    return leaf_node

# Subdivide the node into 8 octants
  half_size = node_size / 2
  child_centers = [
      (node_center[0] - half_size / 2, node_center[1] - half_size / 2, node_center[2] - half_size / 2),
      (node_center[0] + half_size / 2, node_center[1] - half_size / 2, node_center[2] - half_size / 2),
      (node_center[0] - half_size / 2, node_center[1] + half_size / 2, node_center[2] - half_size / 2),
      (node_center[0] + half_size / 2, node_center[1] + half_size / 2, node_center[2] - half_size / 2),
      (node_center[0] - half_size / 2, node_center[1] - half_size / 2, node_center[2] + half_size / 2),
      (node_center[0] + half_size / 2, node_center[1] - half_size / 2, node_center[2] + half_size / 2),
      (node_center[0] - half_size / 2, node_center[1] + half_size / 2, node_center[2] + half_size / 2),
      (node_center[0] + half_size / 2, node_center[1] + half_size / 2, node_center[2] + half_size / 2),
  ]
  print(child_centers)
  tempChildCenter = child_centers


# Check if any child octant has voxel values greater than prediction threshold (0.5)
  child_values = [data[child_center] for child_center in child_centers]
  keptVoxel = any(value > 0.5 for value in child_values)

# Create an internal node or leaf node based on the condition
  current_node = OctreeNode(node_center, node_size)
  if keptVoxel:
    # Create internal node and recursively call for each child
    current_node.children = [
        createOctree(data, level - 1, min_size, child_centers[i], half_size)
        for i in range(8)
    ]
  else:
    # Create leaf node
    current_node.value = max(child_values)

  return current_node

In [ ]:
data = []

for i in range(output.shape[1]):
  for j in range(output.shape[2]):
    for k in range(output.shape[3]):
      data.append({(i, j, k): output[0][i,j,k]})

In [ ]:
print(data[0])
min_size = 0.5
tempVal = [(4.0, 4.0, 4.0), (12.0, 4.0, 4.0), (4.0, 12.0, 4.0), (12.0, 12.0, 4.0), (4.0, 4.0, 12.0)]
data[]
# root = createOctree(data, 3, min_size, (8,8,8), 16)

{(0, 0, 0): tensor(-0.0134, grad_fn=<SelectBackward0>)}
(4.0, 4.0, 4.0)


TypeError: ignored

In [ ]:
'''
  For each voxel, find the density value in each neighboring axis to calculate normal vector
  using central difference method
    Ex: a voxel at position X = 3, Y = 6, Z = 9
    Find value at Grad_X = ((2,6,9) - (4,6,9)) / 2
    Grad_Y = ((3,5,9) - (3,7,9) / 2)
    Grad_Z = ((3,6,8) - (3,6,10) / 2)
    Vector = (Grad_X, Grad_Y, Grad_Z) at (3,6,9)
'''

tempSub = []


def calcCentralDiff(output, i, j, k):
  xLimit = output.shape[1]
  yLimit = output.shape[2]
  zLimit = output.shape[3]

  # output[0] - the first index in output shape corresponds to # of channels so we skip
  output = output[0]
  voxel = output[i,j,k]
  print(i,j,k)

  if(i-1 == -1):
    loweVox_X = 0
  else:
    lowerVox_X = output[i-1, j, k]
  if(i+1 == xLimit):
    upperVox_X = 0
  else:
    upperVox_X = output[i+1, j, k]

  Grad_X = (upperVox_X - lowerVox_X) / 2.0

  if(j-1 == -1):
    lowerVox_Y = 0
  else:
    lowerVox_Y = output[i, j-1, k]
  if(j+1 == yLimit):
    upperVox_Y = 0
  else:
    upperVox_Y = output[i, j+1, k]

  Grad_Y = (upperVox_Y - lowerVox_Y) / 2.0

  if(k-1 == -1):
    lowerVox_Z = 0
  else:
    lowerVox_Z = output[i, j, k-1]
  if(k+1 == zLimit):
    upperVox_Z = 0
  else:
    upperVox_Z = output[i, j, k+1]

  Grad_Z = (upperVox_Z - lowerVox_Z) / 2.0

  vector = [Grad_X, Grad_Y, Grad_Z]
  return vector


for i in range(output.shape[1]):
  for j in range(output.shape[2]):
    for k in range(output.shape[3]):
      normal_vec = calcCentralDiff(output, i,j,k)
      # Sample from vec with SD
      # if(normal_vec[0] + normal_vec[1] + normal_vec[2] > 0.1):
      #   tempSub.append({'i': i, 'j': j, 'k': k, 'vec': normal_vec})
      if(i % 6 == 0 or j % 6 == 0 or k % 6 == 0):
        tempSub.append({'i': i, 'j': j, 'k': k, 'vec': normal_vec})


In [ ]:
print(tempSub)

[{'i': 0, 'j': 0, 'k': 0, 'vec': [tensor(0.0015, grad_fn=<DivBackward0>), tensor(5.4702e-07, grad_fn=<DivBackward0>), tensor(0.0016, grad_fn=<DivBackward0>)]}, {'i': 0, 'j': 0, 'k': 1, 'vec': [tensor(0.0023, grad_fn=<DivBackward0>), tensor(0.0002, grad_fn=<DivBackward0>), tensor(0.0004, grad_fn=<DivBackward0>)]}, {'i': 0, 'j': 0, 'k': 2, 'vec': [tensor(0.0023, grad_fn=<DivBackward0>), tensor(0.0002, grad_fn=<DivBackward0>), tensor(-2.7756e-05, grad_fn=<DivBackward0>)]}, {'i': 0, 'j': 0, 'k': 3, 'vec': [tensor(0.0023, grad_fn=<DivBackward0>), tensor(0.0002, grad_fn=<DivBackward0>), tensor(3.1770e-06, grad_fn=<DivBackward0>)]}, {'i': 0, 'j': 0, 'k': 4, 'vec': [tensor(0.0023, grad_fn=<DivBackward0>), tensor(0.0002, grad_fn=<DivBackward0>), tensor(0., grad_fn=<DivBackward0>)]}, {'i': 0, 'j': 0, 'k': 5, 'vec': [tensor(0.0023, grad_fn=<DivBackward0>), tensor(0.0002, grad_fn=<DivBackward0>), tensor(0., grad_fn=<DivBackward0>)]}, {'i': 0, 'j': 0, 'k': 6, 'vec': [tensor(0.0023, grad_fn=<DivBack